In [1]:
# 我们要定位文件位置，使用这里的代码，具体要将此notebook文件和function文件（或文件夹）放在同一个文件夹路径下，具体路径是什么可以左侧找到目标文件夹然后右击复制文件夹路径来获得
import sys
sys.path.insert(0,'/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes')

In [ ]:
pip install einops

In [ ]:
pip install wandb

In [ ]:
pip install pytorch_lightning

In [ ]:
pip install pykeops

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
import argparse
import json
import numpy as np
#------------------------------------------------------------------
import tensorflow as tf
#import torch
#import torch.nn as nn
#------------------------------------------------------------------
from utils.util import find_max_epoch, print_size, training_loss, calc_diffusion_hyperparams
from utils.util import get_mask_mnr, get_mask_bm, get_mask_rm

from imputers.DiffWaveImputer import DiffWaveImputer
from imputers.SSSDSAImputer import SSSDSAImputer
from imputers.SSSDS4Imputer import SSSDS4Imputer

import easydict

In [15]:
#Check that you have the last version of tensorflow
tf.__version__

'2.9.2'

In [8]:
# We can check the train data's dimensions. It is a 3-D dataset
# Mujoco 数据中，第一维度(0): 8000代表8000个时间序列，或者8000次实验轨道。因为此数据集一共10,000个序列，根据80%作为train数据，20%作为test数据而分配到了8000次实验的train数据
# 第二维度(1): 100表示每次实验中的100个时间点(time points)
# 第三维度(2): 14表示14个features
# 上述分析均来自target paper原文中对Mujoco数据的解释
data = np.load("/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/train_mujoco.npy")
data.shape

(8000, 100, 14)

In [9]:
def train(output_directory,
          ckpt_iter,
          n_iters,
          iters_per_ckpt,
          iters_per_logging,
          learning_rate,
          use_model,
          only_generate_missing,
          masking,
          missing_k):
    
    """
    Train Diffusion Models

    Parameters:
    output_directory (str):         save model checkpoints to this path
    ckpt_iter (int or 'max'):       the pretrained checkpoint to be loaded; 
                                    automatically selects the maximum iteration if 'max' is selected
    data_path (str):                path to dataset, numpy array.
    n_iters (int):                  number of iterations to train
    iters_per_ckpt (int):           number of iterations to save checkpoint, 
                                    default is 10k, for models with residual_channel=64 this number can be larger
    iters_per_logging (int):        number of iterations to save training log and compute validation loss, default is 100
    learning_rate (float):          learning rate

    use_model (int):                0:DiffWave. 1:SSSDSA. 2:SSSDS4.
    only_generate_missing (int):    0:all sample diffusion.  1:only apply diffusion to missing portions of the signal
    masking(str):                   'mnr': missing not at random, 'bm': blackout missing, 'rm': random missing
    missing_k (int):                k missing time steps for each feature across the sample length.
    """

    # generate experiment (local) path
    local_path = "T{}_beta0{}_betaT{}".format(diffusion_config["T"],
                                              diffusion_config["beta_0"],
                                              diffusion_config["beta_T"]) 

    # Get shared output_directory ready
    output_directory = os.path.join(output_directory, local_path)
    if not os.path.isdir(output_directory):
        os.makedirs(output_directory)
        os.chmod(output_directory, 0o775)
    print("output directory", output_directory, flush=True)

    # map diffusion hyperparameters to gpu
    for key in diffusion_hyperparams:
        if key != "T":
            diffusion_hyperparams[key] = diffusion_hyperparams[key].cuda() #[Zihao]: 这里从util.py获得超参数后，对于所有的不是T的超参数(e.g., "Beta","Alpha"),都分配到gpu上，即记为cuda

    # predefine model
    if use_model == 0:
        net = DiffWaveImputer(**model_config).cuda()  # [Zihao]: 这里引入了imputers文件夹内的提前预定好的三个模型：[0] CSDI, [1] SSSDSA [2] sssds4， net:torch network
    elif use_model == 1:                  # 这里的**model_config表示在不同imputers下的提前预定好的模型参数，不同imputer的参数略有不同，可以看
        net = SSSDSAImputer(**model_config).cuda()   # 下面的一个代码块内定义了：global model_config，获得的参数来自各自imputer模型的json结构文件中
    elif use_model == 2:                  # 的 'wavenet_config' (对SSSDSA模型是"sashimi_config")，在下面的代码块内会指定imputer json结构文件的路径来获得参数
        net = SSSDS4Imputer(**model_config).cuda()   # 这里的net是nn.Model,是一种torch模型
    else:
        print('Model chosen not available.')
    print_size(net)

    # <***>define optimizer
    optimizer = tf.keras.optimizers.Adam(net.parameters(), lr=learning_rate)                       #[Zihao]: torch used
                                                            # 这是 Adam算法，一种随机最优化算法，给出可迭代的参数与学习率，进行converge，这里给出的
                                                            # 参数就是net.parameters(), 这是特定imputer模型的初始化参数，net在前面被定义为了一个torch model(已转为cuda),这是pytorch框架下的model,model.parameters()就会返回此model所有参数list
                                                            # lr是Adam算法的学习率参数，我们规定为learning_rate,来自于
                                                            # 对应imputer的json文件中的"train_config"
    # load checkpoint
    if ckpt_iter == 'max':          # 我们在imputer的json文件中设置的就是max，所以checkpoint会首先从我们储存的output_directory中寻找之前存储的检查点，避免花费大量时间继续构建结构
        ckpt_iter = find_max_epoch(output_directory)    # find_max_epoch来自util.py文件里面的class
    if ckpt_iter >= 0:            # 如果找到了存储在output_directory中的检查点：
        try:
            # load checkpoint file
            model_path = os.path.join(output_directory, '{}.pkl'.format(ckpt_iter))
            checkpoint = torch.load(model_path, map_location='cpu')                   # [Zihao]: torch used
                                                            # 这里使用了torch去读取(开包)之前存储的checkpoint,属于pytorch的特定用法
            # feed model dict and optimizer state
            net.load_state_dict(checkpoint['model_state_dict'])                       # 这里也是torch,checkpoint['model_state_dict']里存储着之前生成的模型的已经存了的参数，是一个字典,这里是让net model读取每一个神经网络层的参数, load_state_dict是torch.nn.Module.load_state_dict(),就是为了读取nn模型的参数字典
            if 'optimizer_state_dict' in checkpoint:                           # deserialize 存储在checkpoint中,见下面的torch.save: model_state_dict': net.state_dict()，如果之前没有使用过optimizer模型进行迭代，则初始参数会使用net.state_dict(),
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])              # 当然如果我们之前进行过train的迭代，即使用过optimizer模型(Adam)，那么会产生新的参数，所以我们会使用下面optimizer.state_dict()存储的checkpoint['optimizer_state_dict']

            print('Successfully loaded model at iteration {}'.format(ckpt_iter))           # 到这里才算是将checkpoint里面所有参数重新读取到新的模型中了
        except:
            ckpt_iter = -1
            print('No valid checkpoint model found, start training from initialization try.')   # 如果没有找到checkpoint，意味着要从头训练模型结构，这时可以看到手动将ckpt_iter设定为-1，为的是下面的train过程，不同的ckpt_iter会有不同的结果
    else:
        ckpt_iter = -1
        print('No valid checkpoint model found, start training from initialization.')

        
        
    
    ### Custom data loading and reshaping ###
        
        

    training_data = np.load(trainset_config['train_data_path'])  # 'train_data_path'        # trainset_config 在下面的初始化里面有，表示从imputer的json结构中获得train data的路径
    training_data = np.split(training_data, 160, 0)     # 这个将会把 training_data 分为160个相等分段(batch)，0表现axis=0，沿着横轴(时间)分割，training_data是3-D矩阵数据，第一维度是实验轨道次数：8000(0)，第二维度是每个轨道的100个样本时间点(1), 第三维度是14个features(2),所以这里表示分割为160份数据，features数目保持不变，时间保持不变，轨道分为160份，每份8000/160个轨道实验
    training_data = np.array(training_data)          # 转为array
    training_data = torch.from_numpy(training_data).float().cuda()                        # [Zihao]: torch used
    print('Data loaded')                                             # 这里的含义是将array转为一个pytorch下的tensor(转化为浮点数32),之后声明转化为cuda则表示将此tensor分配到gpu上

    # training
    n_iter = ckpt_iter + 1                 # 这里的train.py的训练过程，如果在上面已经找到了之前训练过的checkpoint,则在上面已经赋给model了checkpoint内的神经网络层的参数，比如SSSDS4模型我们之前设定的"n_iters": 500 (在json文件中)，产生的checkpoint有600.pkl
                                  # 那么这里的ckpt_iter = 600, 则n_iter = 601>下面的n_iters+1 = 501 所以直接不进行while循环，直接跳出不再训练。
                                  # 如果在上面的checkpoint内没有发现.pkl记忆点文件，则上面会赋值ckpt_iter = -1, 此时n_iter = 0,则会进行下面的while循环进行训练，每次训练结束会给n_iter+1,当循环500次的时候，末尾再加1变成n_iter=501 = n_iters + 1(我们设定的),此时跳出循环完成训练
    while n_iter < n_iters + 1:
        for batch in training_data:           # 在上面我们将training_data分为了160组数据，那么应该就是160个batch，进行批量训练

            if masking == 'rm':           # 这里就是看missing的数据类型是什么，可以是'rm': random missing, 'mnr': missing not at random, 'bm': blackout missing, 在SSSDS4里我们目前设定的是rm
                transposed_mask = get_mask_rm(batch[0], missing_k)  # get_mask_rm从util.py中import的,样本为batch[0]，这是随便选的第一个batch为例子，维度只有时间和feature两个, 因为在util.py里面我们只需要用到sample.shape，shape的大小数值而已
            elif masking == 'mnr':
                transposed_mask = get_mask_mnr(batch[0], missing_k)  # get_mask_mnr从util.py中import的,样本为batch[0]，这是随便选的第一个batch为例子，维度只有时间和feature两个, 因为在util.py里面我们只需要用到sample.shape，shape的大小数值而已
            elif masking == 'bm':
                transposed_mask = get_mask_bm(batch[0], missing_k)  # get_mask_bm从util.py中import的,样本为batch[0]，这是随便选的第一个batch为例子，维度只有时间和feature两个, 因为在util.py里面我们只需要用到sample.shape，shape的大小数值而已

            mask = transposed_mask.permute(1, 0)             # transposed_mask是一个torch tensor,这个permute属性是将此tensor的维度转为transposed_mask原始维度(0,1)中，1变到现在的第一号位，0变到现在的第二号位置: (1,0),其实就是行变列，列变行，现在行有14个表示features，列有100个表示每个轨道的时间点
            mask = mask.repeat(batch.size()[0], 1, 1).float().cuda()   # torch tensor.repeat, 表示将原始mask重复，构成新的tensor,结构设定为重复batch第一维度：轨道数，以此来补充满每个batch内的所有轨道数: 8000/160,每列每行中一个元素: 0 or 1，batch是一个三维数组，后两维都repeat了1次说明保持不变，于是这里说明我们直接将mask加了一个维度即batch.size()[0](第一维度)
            loss_mask = ~mask.bool()  #将mask的值都转为bool值，0与1，之后使用~运算符表示：按位取反，本来是0的改为1，本来是1的改为0，之后叫做loss_mask
            batch = batch.permute(0, 2, 1) # 这里表示把batch的维度的后两个维度替换位置，现在第一维度轨道数保持不变，第二维度变成features，第三维度变成每个轨道的时间点数，此时和mask的三个维度重合

            assert batch.size() == mask.size() == loss_mask.size()  # assert是一个检查维护过程，这里检查batch, mask和loss_mask的size(三个维度大小)是否都相等

            # back-propagation
            optimizer.zero_grad()  # 表示在进行后向传播的时候的起点，0 gradient即0梯度
            X = batch, batch, mask, loss_mask  # 这里在输入要计算loss function的自变量X, X[0] = batch, X[1] = batch, X[2] = mask. X[3] = loss_mask
            loss = training_loss(net, nn.MSELoss(), X, diffusion_hyperparams,         # 这里使用的training_loss类在util.py文件中，返回的是loss值，即nn.MSE值，diffusion_hyperparams是计算出来的，来自util.py的calc_diffusion_hyperparams
                                 only_generate_missing=only_generate_missing)  # 注意最后的only_generate_missing，看上面的介绍，这个值如果是0则表示使用所有样本进行diffusion后获得MSE，如果是1则表示只使用missing部分进行diffusion获得的MSE

            loss.backward()    # 目前没发现这里用backward()有什么具体作用，主要作用就是返回loss的值是多少，loss是一个tensor，有值
            optimizer.step()   # optimizer.step() 用于更新所有给定的参数，即给Adam最优化算法的参数： net.parameters()

            if n_iter % iters_per_logging == 0:   # 默认是100，即当迭代次数是整百次的时候，我们这里print出已经进行了多少次迭代和loss值是多少
                print("iteration: {} \tloss: {}".format(n_iter, loss.item()))

            # save checkpoint
            if n_iter > 0 and n_iter % iters_per_ckpt == 0:  # 在进行了整百次训练的时候：
                checkpoint_name = '{}.pkl'.format(n_iter)  #这是在给checkpoint文件(.pkl)进行命名
                torch.save({'model_state_dict': net.state_dict(),  #这是进行checkpoint文件的存储，按照字典的方式，存了未经过训练的初始化参数net.state_dict()，以及经过训练后的基于Adam最优算法迭代后的参数：optimizer.state_dict()
                            'optimizer_state_dict': optimizer.state_dict()},
                           os.path.join(output_directory, checkpoint_name))  # 存储路径为json文件内定义的output_directory，命名为checkpoint的文件名
                print('model at iteration %s is saved' % n_iter)  # print出checkpoint以及完成储存

            n_iter += 1   # 迭代次数+1，进行下一次的迭代

   

In [12]:
if __name__ == "__main__":                  # __name__ == "__main__" 表示目前run的就是train_self_written1_SSSDS4.ipynb文件本身，使用的模型(即train，上面block里定义的)都来自这个文件,这个代码块代表train()模型初始化时候的给定参数
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--config', type=str, default='/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/config/config_SSSDS4.json',  
                        help='JSON file for configuration')  # 这里只是表示读取文件的位置等，当run这个block的时候显示的比较好看

    args = parser.parse_args("")

    with open(args.config) as f:   # 读取json参数内容
        data = f.read()   # 读取json参数内容

    config = json.loads(data)  # 读取json参数内容， 将变量config赋值为json形式下的data
    print(config)   

    train_config = config["train_config"]  # training parameters  #训练模型的参数，来自对应imputer的json参数

    global trainset_config
    trainset_config = config["trainset_config"]  # to load trainset  # train和test数据的路径参数，来自对应imputer的json参数

    global diffusion_config
    diffusion_config = config["diffusion_config"]  # basic hyperparameters  # 这是diffusion模型的初始化超参数，来自对应imputer的json参数，之后会带入util.py文件的calc_diffusion_hyperparams去计算出扩散模型的超参数

    global diffusion_hyperparams
    diffusion_hyperparams = calc_diffusion_hyperparams(
        **diffusion_config)  # dictionary of all diffusion hyperparameters  # 这里就是使用了util.py文件内的calc_diffusion_hyperparams,引入所有diffusion_config内的参数去计算出了扩散模型的超参数

    global model_config
    if train_config['use_model'] == 0:         # 这些是imputer对应模型的初始化超参数，分别对应0:DiffWave. 1:SSSDSA. 2:SSSDS4.
        model_config = config['wavenet_config']   # 给model_config赋值这些参数，之后可以带入模型进行imputer初始化，方便后续使用Adam算法进行迭代，见上面
    elif train_config['use_model'] == 1:
        model_config = config['sashimi_config']
    elif train_config['use_model'] == 2:
        model_config = config['wavenet_config']


{'diffusion_config': {'T': 200, 'beta_0': 0.0001, 'beta_T': 0.02}, 'wavenet_config': {'in_channels': 14, 'out_channels': 14, 'num_res_layers': 36, 'res_channels': 256, 'skip_channels': 256, 'diffusion_step_embed_dim_in': 128, 'diffusion_step_embed_dim_mid': 512, 'diffusion_step_embed_dim_out': 512, 's4_lmax': 100, 's4_d_state': 64, 's4_dropout': 0.0, 's4_bidirectional': 1, 's4_layernorm': 1}, 'train_config': {'output_directory': '/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/Results_SSSDS4/Mujoco/train_90', 'ckpt_iter': 'max', 'iters_per_ckpt': 100, 'iters_per_logging': 100, 'n_iters': 500, 'learning_rate': 0.0002, 'only_generate_missing': 1, 'use_model': 2, 'masking': 'rm', 'missing_k': 90}, 'trainset_config': {'train_data_path': '/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/train_mujoco.npy', 'test_data_path': '/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/test_mujoco.npy', 'segment_length': 100, 'sampling_rate': 100}, 'gen_config': {'output_directory': '/content/drive

In [13]:
# The start of the training.
# All the parameters are the already set ones in **train_config, it is from the json file we set, 
# here is "/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/config/config_SSSDS4.json",
# the "**" means the "train" model will accept all the parameters in the json file above. So by this way,
# all the already defined parameters in the json file will by inputted into the "train" model 

# It will take lots of time when you first run it as it will generate the iteration saving checkpoint. So next
# time when you run it, it will just start from the last iteration memory checkpoint. The path you can find is
# in the json file called: "output_directory"

train(**train_config)

output directory /content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/Results_SSSDS4/Mujoco/train_90/T200_beta00.0001_betaT0.02
SSSDS4Imputer Parameters: 48.371726M
Successfully loaded model at iteration 600
Data loaded


In [ ]:
aaa = train(**train_config)

output directory ./results/mujoco/90/T200_beta00.0001_betaT0.02
SSSDS4Imputer Parameters: 48.371726M
Successfully loaded model at iteration 600
Data loaded


In [ ]:
# Test whether GPU is using
!nvidia-smi

Fri Nov 18 17:23:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.is_available()


True

————————————————————————————————————————

The following is done by myself used for some exercises

————————————————————————————————————————

In [ ]:
import torch

In [ ]:
aaa=torch.tensor([2, 3, 5])

In [ ]:
aaa.repeat(3,5)

tensor([[2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5],
        [2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5],
        [2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5]])

In [ ]:
aaa.repeat(3, 1, 2)

tensor([[[2, 3, 5, 2, 3, 5]],

        [[2, 3, 5, 2, 3, 5]],

        [[2, 3, 5, 2, 3, 5]]])

In [ ]:
aaa.repeat(3, 1, 2).size()

torch.Size([3, 1, 6])

In [ ]:
aaa.repeat(3,5).shape[0]

3

In [ ]:
data = np.load("/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/train_mujoco.npy")

In [ ]:
data

In [ ]:
torch.manual_seed(0)
import torch
import torch.nn as nn
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

In [ ]:
output

tensor(1.6508, grad_fn=<MseLossBackward0>)

In [7]:
import tensorflow as tf
data = (2,3,2,4)
x = tf.random.normal(data[-3:])
x

<tf.Tensor: shape=(3, 2, 4), dtype=float32, numpy=
array([[[-1.8512957 , -0.10105247, -0.01942298, -0.64788115],
        [-1.4454846 , -1.2928028 ,  0.61011493,  0.59740067]],

       [[ 2.7605448 ,  1.2194499 ,  0.9114438 ,  0.97699237],
        [ 0.05329418,  0.71749556,  0.7032246 , -0.11779961]],

       [[ 1.8865706 ,  1.7269831 , -0.06335431, -1.1300508 ],
        [ 1.6945406 ,  0.6119375 , -1.2051554 ,  0.09572101]]],
      dtype=float32)>